<a href="https://colab.research.google.com/github/siting1206/AIcup_NLP/blob/main/2022_AIcup_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch
!pip install pandas nltk
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.1 MB/s 
     |████████████████████████████████| 182 kB 70.8 MB/s 
     |████████████████████████████████| 7.6 MB 55.0 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch, pandas as pd
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
# Training data file
directory="/content/drive/MyDrive/AICUP_NLP/"


file=directory+"Batch_answers - test_data(no_label).csv"
df=pd.read_csv(file, encoding = "ISO-8859-1")
df.head()

,id,q,r,s
0,6199,"""-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- ...","""If so , why do we still have apes , and why a...",DISAGREE
1,5807,"""There 's a lot of discussion there on that is...","""Of course . The makers of Expelled were withi...",DISAGREE
2,8487,"""`` It 's not helping . The guns these people ...","""Oh , I would wager about like Mexico , about ...",DISAGREE
3,1760,"""Shooting : 3 seriously injured in Auburn shoo...","""Pickup strikes group of four youths | Houston...",AGREE
4,6228,"""This is the argument concerning 'choice ' tha...","""I believe there is a point at which we ( soci...",DISAGREE


In [ ]:
df[['q','r']] = df[['q','r']].apply(lambda x: x.str.strip('\"'))
df['r'] = df['s'] + ':' + df['r']
test = df

In [ ]:
test.head()

,id,q,r,s
0,6199,-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -...,"DISAGREE:If so , why do we still have apes , a...",DISAGREE
1,5807,There 's a lot of discussion there on that iss...,DISAGREE:Of course . The makers of Expelled we...,DISAGREE
2,8487,`` It 's not helping . The guns these people h...,"DISAGREE:Oh , I would wager about like Mexico ...",DISAGREE
3,1760,Shooting : 3 seriously injured in Auburn shoot...,AGREE:Pickup strikes group of four youths | Ho...,AGREE
4,6228,This is the argument concerning 'choice ' that...,DISAGREE:I believe there is a point at which w...,DISAGREE


## Tokenizer

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
test_data_q = df['q'].tolist()
test_data_r = df['r'].tolist()
test_encodings = tokenizer(test_data_q, test_data_r, truncation=True, padding=True)
tokenizer.decode(test_encodings['input_ids'][0])

'[CLS] - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - ( Time. ) [SEP] DISAGREE : If so, why do we still have apes, and why are there no transitional species that we can currently observe. Statistical averages say we should see a few hundred within the last two centuries, at a minimum. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

## Dataset

In [ ]:
class qrDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
test_dataset = qrDataset(test_encodings)

In [ ]:
next(iter(test_dataset))

{'input_ids': tensor([  101,   118,   118,   118,   118,   118,   118,   118,   118,   118,
           118,   118,   118,   118,   118,   118,   118,   118,   118,   118,
           118,   118,   118,   118,   118,   118,   118,   118,   118,   118,
           118,   118,   118,   118,   118,   118,   118,   118,   118,   118,
           118,   118,   118,   118,   118,   118,   118,   118,   118,   118,
           118,   118,   118,   118,   118,   118,   118,   118,   118,   118,
           118,   118,   118,   118,   118,   118,   118,   118,   118,   118,
           118,   118,   118,   118,   118,   113,  2614,   119,   114,   102,
           141,  6258, 22689, 16941,  2036,   131,  1409,  1177,   117,  1725,
          1202,  1195,  1253,  1138,   170,  6633,   117,  1105,  1725,  1132,
          1175,  1185, 23512,  1530,  1115,  1195,  1169,  1971, 12326,   119,
         12121, 22284,  1474,  1195,  1431,  1267,   170,  1374,  2937,  1439,
          1103,  1314,  1160,  3944,   

## Model

In [ ]:
from transformers import BertModel

class myModel(torch.nn.Module):

    def __init__(self):

        super(myModel, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.fc = nn.Linear(768, 4)
        

    def forward(self, input_ids, attention_mask, token_type_ids):

        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, return_dict=True)
        logits = output[0]
        out = self.fc(logits)

        return out

In [ ]:
from transformers import AdamW
from tqdm import tqdm

# Set GPU / CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# Put model on device
model = myModel().to(device)

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
batch_size = 8
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model.load_state_dict(torch.load(directory + 'aicup_model'))

<All keys matched successfully>

## Predict

In [ ]:
def predict(test_loader):
    predict_pos = []

    model.eval()

    q_sub_output, r_sub_output = [],[]

    loop = tqdm(test_loader, leave=True)
    for batch_id, batch in enumerate(loop):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)

        # model output
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        
        q_start_logits, r_start_logits, q_end_logits, r_end_logits = torch.split(outputs, 1, 2)

        q_start_logits = q_start_logits.squeeze(-1).contiguous()
        r_start_logits = r_start_logits.squeeze(-1).contiguous()
        q_end_logits = q_end_logits.squeeze(-1).contiguous()
        r_end_logits = r_end_logits.squeeze(-1).contiguous()

        q_start_prdict = torch.argmax(q_start_logits, 1).cpu().numpy()
        r_start_prdict = torch.argmax(r_start_logits, 1).cpu().numpy()
        q_end_prdict = torch.argmax(q_end_logits, 1).cpu().numpy()
        r_end_prdict = torch.argmax(r_end_logits, 1).cpu().numpy()

        for i in range(len(input_ids)):
            predict_pos.append((q_start_prdict[i].item(), r_start_prdict[i].item(), q_end_prdict[i].item(), r_end_prdict[i].item()))

            q_sub = tokenizer.decode(input_ids[i][q_start_prdict[i]:q_end_prdict[i]+1])
            r_sub = tokenizer.decode(input_ids[i][r_start_prdict[i]:r_end_prdict[i]+1])
            
            q_sub_output.append(q_sub)
            r_sub_output.append(r_sub)
    
    return q_sub_output, r_sub_output, predict_pos

In [ ]:
q_sub_output, r_sub_output, predict_pos = predict(test_loader)

100%|██████████| 252/252 [01:11<00:00,  3.52it/s]


In [ ]:
def get_output_post_fn(test, q_sub_output, r_sub_output):
    q_sub, r_sub = [], []
    for i in range(len(test)):

        q_sub_pred = q_sub_output[i].split()
        r_sub_pred = r_sub_output[i].split()

        if q_sub_pred is None:
            q_sub_pred = []
        q_sub_error_index = q_sub_pred.index('[SEP]') if '[SEP]' in q_sub_pred else -1

        if q_sub_error_index != -1:
            q_sub_pred = q_sub_pred[:q_sub_error_index]

        temp = r_sub_pred.copy()
        if r_sub_pred is None:
            r_sub_pred = []
        else:
            for j in range(len(temp)):
                if temp[j] == '[SEP]':
                    r_sub_pred.remove('[SEP]')
                if temp[j] == '[PAD]':
                    r_sub_pred.remove('[PAD]')

        q_sub.append(' '.join(q_sub_pred))
        r_sub.append(' '.join(r_sub_pred))

    return q_sub, r_sub

In [ ]:
q_sub, r_sub = get_output_post_fn(test, q_sub_output, r_sub_output)

In [ ]:
test['q_sub'] = q_sub
test['r_sub'] = r_sub

In [ ]:
test.head()

,id,q,r,s,q_sub,r_sub
0,6199,-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -...,"DISAGREE:If so , why do we still have apes , a...",DISAGREE,- - - - - - - - - - - - - - - - - - - - - - - ...,"If so, why do we still have apes, and why are ..."
1,5807,There 's a lot of discussion there on that iss...,DISAGREE:Of course . The makers of Expelled we...,DISAGREE,There's a lot of discussion there on that issu...,Of course. The makers of Expelled were within ...
2,8487,`` It 's not helping . The guns these people h...,"DISAGREE:Oh , I would wager about like Mexico ...",DISAGREE,"The guns these people have, they don't registe...","Oh, I would wager about like Mexico, about 80 ..."
3,1760,Shooting : 3 seriously injured in Auburn shoot...,AGREE:Pickup strikes group of four youths | Ho...,AGREE,Shooting : 3 seriously injured in Auburn shoot...,Pickup strikes group of four youths | Houston ...
4,6228,This is the argument concerning 'choice ' that...,DISAGREE:I believe there is a point at which w...,DISAGREE,"This is the argument concerning'choice'that, t...",I believe there is a point at which we ( socie...


In [ ]:
test.drop(['q','r','s'], axis=1)
test.head()

In [ ]:
test.to_csv("submission.csv")